In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
api_key="your OPENAI API key"

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


# Quantitative benchmark

## Paper 1 benchmark

In [ ]:
# Replace 'path_to_your_file.txt' with the actual path of your file
file_path = '/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Raw Data/paper_1_hl.txt'

# Reading the file content
with open(file_path, 'r') as file:
    content = file.read()

# Splitting the content by '###'
split_content = content.split('###')

# 'split_content' is now a list where each element is a portion of the original string separated by '###'


In [ ]:
len(split_content)

653

In [ ]:
split_content[7]

'\n\nA red blood cell is 8 μm in diameter. If drawn 100 times larger than its actual size, what diameter will the drawing be in mm?\n\nA. 0.08 mmB. 0.8 mmC. 8 mmD. 80 mm\n\nMarkscheme\n\nB\n\nExaminers report\n\nMany candidates were not able to do this calculation correctly and convert µm to mm.\xa0 Although the size of the drawing calculated was admittedly too small for a real drawing, there was no reason why the correct conversion could not be made. It did discriminate as the more able candidates got this correct while the others guessed between the other choices of answers.\xa0\xa0\n\n\n\n'

In [ ]:


for i in range(len(split_content)):
    # Split the string at "Examiners report" and take the first part
    split_string = split_content[i].split("Examiners report")
    split_content[i] = split_string[0]

# Now split_content contains strings with everything before "Examiners report"
split_content[7]

'\n\nA red blood cell is 8 μm in diameter. If drawn 100 times larger than its actual size, what diameter will the drawing be in mm?\n\nA. 0.08 mmB. 0.8 mmC. 8 mmD. 80 mm\n\nMarkscheme\n\nB\n\n'

In [ ]:
from openai import OpenAI
import os
client = OpenAI(api_key=api_key)

In [ ]:
def classify(question):


    prompt = (

        f"""Respond with 0 if the question refers to an image or demands a mathematical calculation, and a 1 otherwise.
        For example:
        Question: Which organ is label X?
        A. stomach
        B. kidney
        C. intestine
        D. Bladder
        Markscheme: C

        Classification:0

        Now do the same for the following:
        Question: {question}


        Classification:"""
    )


    # Use OpenAI API to generate the email response
    response = client.chat.completions.create(
        model= "gpt-4", #gpt-3.5-turbo
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that classifies whether a question refers to an image or not."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,
        max_tokens=3,
        top_p=1
    )
    complete_response = response.choices[0].message.content

    # Return the generated response
    return complete_response.strip()


In [ ]:
def clean_question(question):
    example_input = "A red blood cell is 8 μm in diameter. If drawn 100 times larger than its actual size, what diameter will the drawing be in mm?\n\nA. 0.08 mmB. 0.8 mmC. 8 mmD. 80 mm\n\nMarkscheme\n\nB\n"
    example_output = "Question: A red blood cell is 8 μm in diameter. If drawn 100 times larger than its actual size, what diameter will the drawing be in mm?\n\nA. 0.08 mm\nB. 0.8 mm\nC. 8 mm\nD. 80 mm\n\nMarkscheme: B"

    prompt = (
        "Given the following example of cleaning and formatting a question:\n\n"
        f"Input question: {example_input}\n\n"
        f"Question: {example_output}\n\n"
        "Now, clean and format this new question in the same way by removing the 'examiner report', removing any unecessary characters, formatting it neatly, and retaining the exact same question in terms of content, and the correct mark scheme answer:\n\n"
        f"Input question: {question}"
    )

    # Assuming 'client' is your OpenAI API client already initialized
    response = client.chat.completions.create(
        model="gpt-4",  # Adjust according to your preference or available models
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that cleans and formats examination questions based on examples."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,
        max_tokens=1024,  # Adjusted to ensure enough space for responses
        top_p=1
    )

    complete_response = response.choices[0].message.content

    # Extract and return the cleaned and formatted question
    return complete_response.strip()  # Using strip() to clean up any leading/trailing whitespace


In [ ]:
split_content[6]

'\n\nAn unknown cell is observed using a microscope. A cell wall, ribosomes and DNA are identified. What can be concluded from these observations?\n\nA.\xa0\xa0 \xa0It can only be a prokaryotic cell.B.\xa0\xa0 \xa0It can only be a eukaryotic cell.C.\xa0\xa0 \xa0It could be a prokaryotic or eukaryotic cell.D.\xa0\xa0 \xa0It can only be a plant cell.\n\nMarkscheme\n\nC\n\n'

In [ ]:
clean_question(split_content[8])

'Question: What is proportional to a cell’s surface area?\n\nA. Rate of exchange of materials\nB. Rate of heat production\nC. Rate of waste production\nD. Rate of oxygen consumption\n\nMarkscheme: A'

In [ ]:
classify(split_content[6])

'1'

In [ ]:
for i in range(len(split_content) - 1, -1, -1):
    # Remove leading and trailing whitespace from the string
    trimmed_string = split_content[i].strip()
    # Check if the string is empty or contains only newline characters
    if not trimmed_string or trimmed_string == '\n':
        # Remove the string from the list
        del split_content[i]

In [ ]:
len(split_content)


641

In [ ]:
from tqdm import tqdm
cleaned_data = [clean_question(item) for item in tqdm(split_content, position=0, leave=True)]

100%|██████████| 641/641 [46:19<00:00,  4.34s/it]


In [ ]:
from tqdm import tqdm
classified_data = [int(classify(item)) for item in tqdm(cleaned_data, position=0, leave=True)]

100%|██████████| 641/641 [06:04<00:00,  1.76it/s]


In [ ]:
# Create the DataFrame
df_questions = pd.DataFrame({
    'Raw question': split_content,
    'Clean question': cleaned_data,
    'Suitable question Classification': classified_data
})

# Display the DataFrame
display(df_questions)

,Raw question,Clean question,Suitable question Classification
0,﻿,It seems like you didn't provide a new questio...,1
1,\n\n\n\n\n\n\nIMAGE\n\nHL Paper 1\n\nWhich fun...,Question: Which functions are carried out by a...,1
2,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,1
3,\n\nWhat distinguishes prokaryotic cells from ...,Question: What distinguishes prokaryotic cells...,1
4,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,1
...,...,...,...
636,\n\nThe diagram below shows some stages in the...,Question: The diagram below shows some stages ...,0
637,\n\nWhich letter correctly identifies the medu...,Question: Which letter correctly identifies th...,0
638,\n\nWhat structure is indicated by the arrows?...,Question: What structure is indicated by the a...,0
639,\n\nThe images below show muscle tissue.\n\n\n...,Question: Which image shows contracted muscle ...,0


In [ ]:
df_questions.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 1/p1_formatted.csv',index=False)

In [ ]:
df_questions = df_questions[(df_questions['Suitable question Classification'] == 1) &
                           (df_questions['Clean question'].str.startswith('Question:'))].reset_index(drop=True)
df_questions = df_questions[df_questions['Clean question'].str.contains('Question') &
                          df_questions['Clean question'].str.contains('Markscheme')].reset_index(drop=True)

df_questions

,Raw question,Clean question,Suitable question Classification
0,\n\n\n\n\n\n\nIMAGE\n\nHL Paper 1\n\nWhich fun...,Question: Which functions are carried out by a...,1
1,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,1
2,\n\nWhat distinguishes prokaryotic cells from ...,Question: What distinguishes prokaryotic cells...,1
3,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,1
4,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,1
...,...,...,...
492,\n\nWhat are the roles of the following struct...,Question: What are the roles of the following ...,1
493,\n\nIn which part of the nephron is salt secre...,Question: In which part of the nephron is salt...,1
494,\n\nThe table shows solute concentrations in n...,Question: The table shows solute concentration...,1
495,"\n\nWhere are structures I, II and III found i...","Question: Where are structures I, II and III f...",1


In [ ]:
def classify_suitable_mcq(question):


    prompt = (
        f"""Classification Rules:
        - Assign a classification of 1 if the provided text matches the specified pattern.
        - Assign a classification of 0 if the provided text does not match the specified pattern.

        Determine if the following text follows the specified pattern:
        'Question: What is the correct option?

        A. Wrong
        B. Correct
        C. Wrong
        D. Wrong

        Markscheme: B'

        Classification:1

        Determine if the following text follows the specified pattern:
        'Question: What is the correct option?

        Markscheme: B'
        Classification:0

        Determine if the following text follows the specified pattern:
        'Question: What is the correct option?
        A. Wrong
        B. Correct
        C. Wrong
        D. Wrong

        Markscheme: B, C'
        Classification:0

        Now do the same for the following text:
        '{question}'
        Classification:"""
    )



    # Use OpenAI API to generate the email response
    response = client.chat.completions.create(
        model= "gpt-4", #gpt-3.5-turbo
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that classifies whether a question follows a pattern or not."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,
        max_tokens=3,
        top_p=1
    )
    complete_response = response.choices[0].message.content

    # Return the generated response
    return complete_response.strip()


In [ ]:
classify_suitable_mcq(filtered_df['Clean question'][3])

'1'

In [ ]:
classify_suitable_mcq(filtered_df['Clean question'][0])


'0'

In [ ]:
suitible_question_benchmarking=[int(classify_suitable_mcq(question)) for question in tqdm(df_questions['Clean question'], position=0, leave=True)]

100%|██████████| 497/497 [04:26<00:00,  1.86it/s]


In [ ]:
df_questions['Suitible question for benchmarking']=suitible_question_benchmarking

In [ ]:
df_questions=df_questions[df_questions['Suitible question for benchmarking'] == 1].reset_index(drop=True)

In [ ]:
df_questions

,Raw question,Clean question,Suitable question Classification,Suitible question for benchmarking
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,1,1
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,1,1
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,1,1
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,1,1
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,1,1
...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,1,1
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,1,1
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,1,1
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,1,1


In [ ]:
df_questions = df_questions.rename(columns={'Clean question': 'Formatted Question and Markscheme'})

df_questions = df_questions[['Raw question', 'Formatted Question and Markscheme']]
# Verify the change by displaying the DataFrame's columns
print(df_questions.columns)


Index(['Raw question', 'Formatted Question and Markscheme'], dtype='object')


In [ ]:
import pandas as pd
import re

def format_question(string_quest):
    # Using regular expression to capture everything between "Question:" and "Markscheme:"
    match = re.search(r'Question:(.*?)Markscheme:', string_quest, re.S)
    if match:
        # Extracting the matched group and stripping whitespace
        clean_question_string = match.group(1).strip()
    else:
        # If the pattern is not found, return an empty string or some placeholder
        clean_question_string = '###'
    return clean_question_string

# Apply the function to every element in the 'Formatted Question and Markscheme' column
formatted_questions = df_questions['Formatted Question and Markscheme'].apply(format_question)

# Put the results in a new column called 'Formatted Question'
df_questions['Formatted Question'] = formatted_questions

def format_markscheme(string_quest):
    # Using regular expression to find everything after "Markscheme:"
    match = re.search(r'Markscheme:(.*)', string_quest, re.S)
    if match:
        # Extracting the matched group and stripping whitespace
        clean_markscheme_string = match.group(1).strip()
    else:
        # If the pattern is not found, return an empty string or some placeholder
        clean_markscheme_string = '###'
    return clean_markscheme_string

# Apply the function to every element in the 'Formatted Question and Markscheme' column
formatted_markschemes = df_questions['Formatted Question and Markscheme'].apply(format_markscheme)

# Put the results in a new column called 'Formatted Markscheme'
df_questions['Markscheme'] = formatted_markschemes

# Verify the changes
display(df_questions.head())

In [ ]:
df_questions

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C
...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B


In [ ]:
# Check if '###' is found in either 'Formatted Question' or 'Formatted Markscheme'
contains_hashes = df_questions['Formatted Question'].str.contains('###') | df_questions['Markscheme'].str.contains('###')

# Filter df_questions to show rows where either column contains '###'
df_with_hashes = df_questions[contains_hashes]

# Display rows where '###' is found
display(df_with_hashes)


,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme


In [ ]:
df_questions = df_questions.rename(columns={'Question': 'Raw Question'})

# Verify the change by displaying the DataFrame's columns
print(df_questions.columns)


Index(['Raw question', 'Formatted Question and Markscheme',
       'Formatted Question', 'Markscheme'],
      dtype='object')


In [ ]:
df_questions.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 1/p1_400q_eval_ds.csv',index=False)

In [ ]:
import pandas as pd
df_questions=pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 1/p1_400q_eval_ds.csv')
df_questions

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C
...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B


In [ ]:
df_questions['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
df_questions['Markscheme'].tolist()

# Qualitative Benchmark

## Paper 2

Clean and format the Paper 2 data

In [ ]:
# Define the file path
file_path = '/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Raw Data/paper_2_hl.txt'

# Read the contents of the file

with open(file_path, 'r') as file:
    raw_p2_content = file.read()

In [ ]:
# Split the string into a list at each '###'
raw_p2_content = raw_p2_content.split('###')

# Now split_content is a list where each element is a portion of the string separated by ###
print(len(raw_p2_content))

341


In [ ]:
# Create a new list to store the modified strings
modified_content = []

for string in raw_p2_content:
    # Find the position of "Examiners report" in the string
    index = string.find("Examiners report")

    # If "Examiners report" is found, slice the string up to that point
    if index != -1:
        modified_string = string[:index]
    else:
        # If "Examiners report" is not found, use the whole string
        modified_string = string
    # Append the modified string to the new list
    modified_content.append(modified_string)

raw_p2_content = modified_content

# Print the length of the modified list to verify the changes
print(len(raw_p2_content))

341


In [ ]:
# Filter the list to remove elements with fewer than 10 characters
raw_p2_content = [s for s in raw_p2_content if len(s) >= 10]

# Now filtered_content contains only elements that are 10 characters or longer
print(len(raw_p2_content))


327


In [ ]:
print(raw_p2_content[2])



Explain how minerals move into plants.

[8]

a.

Outline the conditions needed for the germination of a typical seed.

[3]

b.

Following germination of seeds, plants undergo a rapid increase in the number of cells. Describe stages in the cell cycle that result in this increase of cells.

[7]

c.

Markscheme

Remember, up to TWO “quality of construction” marks per essay.

a. minerals bound to soil particles;b. examples of three nutrients from: phosphate, nitrate, magnesium, iron, calcium, potassium, sodium, magnesium;c. minerals dissolve in water;d. mass flow causes movement of minerals with movement of water through soil;e. minerals diffuse down a concentration gradient towards roots (as the mineral concentration next to the roots is continuously decreasing);f. minerals enter the plant through roots;g. by active transport / use of ATP;h. branching of roots increases surface area for absorption of minerals;i. root hairs increase surface area (for the absorption of minerals);j. hypha 

In [ ]:
example_user_input="""
Describe the characteristics of stem cells that make them potentially useful in medicine.

[5]

a.

Outline a technique of gene transfer resulting in genetically modified organisms.

[5]

b.

Explain the use of karyotyping in human genetics.

[8]

c.

Markscheme

(stem cells) have/retain the capacity to divide;can be used to produce cell cultures/large number of identical cells;can be used to repair/replace damaged/lost cells/tissue;(stem cells) are undifferentiated / have not yet differentiated/specialized;can differentiate/specialize in different ways / are pluripotent/totipotent;can be used to form a variety of different tissues / form organs;used in medical research;used in treatment of (named) disease;

a.

gene transfer takes a gene from one species/organism and inserts it into another;using plasmid/viral vector/ballistic impregnation/electroporation;use of reverse transcriptase to obtain gene from mRNA;restriction enzyme/endonuclease used to cut out/excise gene;(same) restriction enzyme used to cut open plasmid;sticky ends used to link DNA/link gene to plasmid;DNA ligase used to seal nicks/splice;bacterium takes in plasmid / plasmid transferred to bacterium/plant/host cell;valid documented example (e.g. human insulin from bacterium/yeast / salt-tolerant tomato plant / carotene/vitamin A in rice /herbicide/Roundup/glyphosate resistance in crop plants / factor IX/clotting factor in sheep milk / low phosphate feces in pigs;

b.

Definition and construction of karyotypes:karyotype is the number and type / image of chromosomes in a cell;cells collected from chorionic villus / by amniocentesis;requires cells in metaphase / stimulate cells to divide and reach metaphase;burst cells and spread chromosomes / photo taken of chromosomes;chromosomes are arranged in pairs;according to size/structure/position of centromere/banding pattern;

Uses for karyotypes:karyotypes used to identify sex/gender;male is XY and female XX;used to identify chromosome mutations/abnormal numbers/non-disjunction;Down syndrome due to extra chromosome 21 / other trisomy/aneuploidy example;used for pre-natal diagnosis of chromosome abnormalities;may lead to a decision to abort the fetus;prepare for consequences of abnormality in offspring;

c.

"""
user_input_formatted="""
Question: Describe the characteristics of stem cells that make them potentially useful in medicine. [5 marks]
Markscheme:
- (stem cells) have/retain the capacity to divide
- can be used to produce cell cultures/large number of identical cells
- can be used to repair/replace damaged/lost cells/tissue;(stem cells) are undifferentiated / have not yet differentiated/specialized
- can differentiate/specialize in different ways / are pluripotent/totipotent;can be used to form a variety of different tissues / form organs
- used in medical research
- used in treatment of (named) disease;
###
Question: Outline a technique of gene transfer resulting in genetically modified organisms. [5 marks]
Markcheme:
- gene transfer takes a gene from one species/organism and inserts it into another
- using plasmid/viral vector/ballistic impregnation/electroporation
- use of reverse transcriptase to obtain gene from mRNA
- restriction enzyme/endonuclease used to cut out/excise gene
- (same) restriction enzyme used to cut open plasmid
- sticky ends used to link DNA/link gene to plasmid
- DNA ligase used to seal nicks/splice
- bacterium takes in plasmid / plasmid transferred to bacterium/plant/host cell
- valid documented example (e.g. human insulin from bacterium/yeast / salt-tolerant tomato plant / carotene/vitamin A in rice /herbicide/Roundup/glyphosate resistance in crop plants / factor IX/clotting factor in sheep milk / low phosphate feces in pigs
###
Question: Explain the use of karyotyping in human genetics. [8 marks]
Markscheme:
- Definition and construction of karyotypes:
- karyotype is the number and type / image of chromosomes in a cell;cells collected from chorionic villus / by amniocentesis;requires cells in metaphase / stimulate cells to divide and reach metaphase
- burst cells and spread chromosomes / photo taken of chromosomes
- chromosomes are arranged in pairs;according to size/structure/position of centromere/banding pattern
- Uses for karyotypes:karyotypes used to identify sex/gender
- male is XY and female XX
- used to identify chromosome mutations/abnormal numbers/non-disjunction
- Down syndrome due to extra chromosome 21 / other trisomy/aneuploidy example
- used for pre-natal diagnosis of chromosome abnormalities
- may lead to a decision to abort the fetus
- prepare for consequences of abnormality in offspring;
"""

In [ ]:
from openai import OpenAI
import os
client = OpenAI(api_key=api_key)

In [ ]:
import os
from openai import OpenAI
from tqdm import tqdm

def generate_q_and_a_from_p2(user_input):
    prompt = f'''
    **Task Description:** Your job is to format a series of questions and their corresponding answers into a standardized format. Do this using the example below:

    **Example Input:**

    {example_user_input}


    **Expected Output:**

    {user_input_formatted}


    **Your Task:** Apply the same formatting to the following input.

    **Input:**

    {user_input}

    **Output:**

    '''

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",#"gpt-4-turbo",#"gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You an AI that formats a series of questions and their corresponding answers into a standardized format"
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.7,
            max_tokens=2000,
            top_p=1
        )
        complete_response = response.choices[0].message.content
        return complete_response
    except Exception as e:
        return f"Error generating response: {e}"


In [ ]:
example=generate_q_and_a_from_p2(raw_p2_content[2])

In [ ]:
print(example)

Question: Explain how minerals move into plants. [8 marks]
Markscheme:
- minerals bound to soil particles
- examples of three nutrients from: phosphate, nitrate, magnesium, iron, calcium, potassium, sodium, magnesium
- minerals dissolve in water
- mass flow causes movement of minerals with movement of water through soil
- minerals diffuse down a concentration gradient towards roots (as the mineral concentration next to the roots is continuously decreasing)
- minerals enter the plant through roots
- by active transport / use of ATP
- branching of roots increases surface area for absorption of minerals
- root hairs increase surface area (for the absorption of minerals)
- hypha of (mutualistic) fungi may enhance movement of selected ions into roots / increase surface area
- root hairs have many mitochondria to provide energy/ATP for active transport
- export of H+ creates electrochemical gradient / displaces ions bound to soil/clay
- that causes positive mineral ions to diffuse into (root

In [ ]:
from tqdm import tqdm
clean_p2_content= [generate_q_and_a_from_p2(q_m) for q_m in tqdm(raw_p2_content)]


100%|██████████| 327/327 [33:16<00:00,  6.10s/it]


In [ ]:
import pandas as pd
# Creating a DataFrame from the list of responses
df_p2 = pd.DataFrame(clean_p2_content, columns=["p2 question and answers"])

df_p2.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/cleaned_p2_q_m.csv',index=False)

In [ ]:
import pandas as pd
df_p2= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/cleaned_p2_q_m.csv')
df_p2

,p2 question and answers
0,Question: Outline the various stages of the ce...
1,Question: Describe the characteristics of stem...
2,Question: Explain how minerals move into plant...
3,Question: Draw a labelled diagram to show the ...
4,Question: Draw a labelled diagram of the ultra...
...,...
322,Question: Label the structures indicated on th...
323,Question: State when PAN was injected into the...
324,Question: Describe the origin of eukaryotic ce...
325,Question: Deduce whether the excretion of ammo...


In [ ]:
def clean_split(string):
    # Split the string by '###'
    split_list = string.split('###')
    # Filter out any empty or whitespace-only elements
    return [item for item in split_list if item.strip()]


In [ ]:
clean_split(df_p2['p2 question and answers'][0])

['Question: Outline the various stages of the cell cycle. [4 marks]\nMarkscheme:\n- G1 the cell grows/duplication of organelles\n- S is synthesis stage when DNA is synthesized/replicated\n- G2 the chromosomes begin condensing/preparation for cell division\n- G1, S and G2 make up interphase\n- during mitosis nuclear division occurs/all four stages listed\n- during cytokinesis cytoplasm/cell divides/daughter cells formed\n',
 '\n\nQuestion: Define the term transpiration and explain the factors that can affect transpiration in a typical terrestrial plant. [9 marks]\nMarkscheme:\n- (transpiration is) loss of water vapour from the leaves/stomata (and stems) of plants\n- temperature, humidity, light (intensity) and wind all affect transpiration\n- high temperatures increase evaporation rate of water/transpiration\n- (accept converse) high humidity lowers the rate of water evaporation/transpiration\n- (accept converse) air currents/wind increase water evaporation/transpiration\n- (accept conv

In [ ]:
individual_p2_questions=[]

for group_of_questions in df_p2['p2 question and answers'].tolist():
    individual_p2_questions+=clean_split(group_of_questions)

In [ ]:
len(individual_p2_questions)

1183

In [ ]:
df_p2_clean = pd.DataFrame(individual_p2_questions, columns=["p2 question and answers"])

In [ ]:
df_p2_clean['p2 question and answers'][0]

'Question: Outline the various stages of the cell cycle. [4 marks]\nMarkscheme:\n- G1 the cell grows/duplication of organelles\n- S is synthesis stage when DNA is synthesized/replicated\n- G2 the chromosomes begin condensing/preparation for cell division\n- G1, S and G2 make up interphase\n- during mitosis nuclear division occurs/all four stages listed\n- during cytokinesis cytoplasm/cell divides/daughter cells formed\n'

In [ ]:
# Define a function to split "Question" from "Markscheme" and remove labels
def split_question_markscheme(text):
    # Split the text into question and markscheme parts
    parts = text.split("Markscheme:")
    question_part = parts[0].replace("Question:", "").strip()
    markscheme_part = parts[1].strip() if len(parts) > 1 else ""

    return pd.Series([question_part, markscheme_part])


In [ ]:
print(split_question_markscheme(df_p2_clean['p2 question and answers'][0])[0])
print(split_question_markscheme(df_p2_clean['p2 question and answers'][0])[1])

Outline the various stages of the cell cycle. [4 marks]
- G1 the cell grows/duplication of organelles
- S is synthesis stage when DNA is synthesized/replicated
- G2 the chromosomes begin condensing/preparation for cell division
- G1, S and G2 make up interphase
- during mitosis nuclear division occurs/all four stages listed
- during cytokinesis cytoplasm/cell divides/daughter cells formed


In [ ]:
# Apply the function and assign results to new columns
df_p2_clean[['Question', 'Markscheme']] = df_p2_clean['p2 question and answers'].apply(split_question_markscheme)

# Display the updated DataFrame
display(df_p2_clean)

,p2 question and answers,Question,Markscheme
0,Question: Outline the various stages of the ce...,Outline the various stages of the cell cycle. ...,- G1 the cell grows/duplication of organelles\...
1,\n\nQuestion: Define the term transpiration an...,Define the term transpiration and explain the ...,- (transpiration is) loss of water vapour from...
2,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...
3,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...
4,\n\nQuestion: Explain the use of karyotyping i...,Explain the use of karyotyping in human geneti...,- Definition and construction of karyotypes:\n...
...,...,...,...
1178,\nQuestion: The salt marshes where these turtl...,The salt marshes where these turtles live peri...,- Problems:\n - urea becomes more concentra...
1179,Question: Outline the process of in vitro fert...,Outline the process of in vitro fertilization ...,- mother receives hormone treatment/FSH to sti...
1180,\n\nQuestion: Identify the cell labelled X. [1...,Identify the cell labelled X. [1 mark],- Sertoli cell / nurse cell
1181,\n\nQuestion: Outline the function of this cel...,Outline the function of this cell. [1 mark],- nourishes maturing sperm(atozoa) / protects ...


In [ ]:
import pandas as pd

df_p2_clean.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/final_p2_q_m.csv',index=False)

In [ ]:
df_p2_clean=pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/final_p2_q_m.csv')
df_p2_clean=df_p2_clean.drop_duplicates(subset=['Question']).reset_index(drop=True)


In [ ]:
df_p2_clean= df_p2_clean.dropna()


In [ ]:
df_p2_clean = df_p2_clean.reset_index(drop=True)

In [ ]:
df_p2_clean.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/final_p2_q_m.csv',index=False)



## Create qualitative benchmark dataset

In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 2/final_p2_q_m.csv')

In [ ]:
df

,p2 question and answers,Question,Markscheme
0,Question: Outline the various stages of the ce...,Outline the various stages of the cell cycle. ...,- G1 the cell grows/duplication of organelles\...
1,\n\nQuestion: Define the term transpiration an...,Define the term transpiration and explain the ...,- (transpiration is) loss of water vapour from...
2,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...
3,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...
4,\n\nQuestion: Explain the use of karyotyping i...,Explain the use of karyotyping in human geneti...,- Definition and construction of karyotypes:\n...
...,...,...,...
606,\n\nQuestion: State the increase in protein in...,State the increase in protein in the urine of ...,- 205 mg day-1 (units required)\n- Allow answe...
607,\n\nQuestion: Compare the levels of protein du...,Compare the levels of protein during the exper...,- on Day 3 little/no difference / both levels ...
608,\n\nQuestion: Evaluate whether the results sup...,Evaluate whether the results support the hypot...,- lower (increase in) protein/greater reductio...
609,\n\nQuestion: Analyse the results of this expe...,Analyse the results of this experiment. [3 marks],- PAN increases TBArs levels / TBArs levels hi...


In [ ]:
import re
pattern = r'\[\s*(2|3)\s*marks\]'

# Filter rows where the 'Question' column does not match the pattern
filtered_df = df[~df['Question'].str.contains(pattern, regex=True)]

# Regular expression to find the pattern for '1 mark' or '1 marks'
pattern_1 = r'\[\s*1\s*marks?\]'

# Filter rows where the 'Question' column does not match the pattern
filtered_df = filtered_df[~filtered_df['Question'].str.contains(pattern_1, regex=True)]

# Reset the index
filtered_df.reset_index(drop=True, inplace=True)

df=filtered_df
df

<ipython-input-6-3e46ec4b0887>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[~df['Question'].str.contains(pattern, regex=True)]


,p2 question and answers,Question,Markscheme
0,Question: Outline the various stages of the ce...,Outline the various stages of the cell cycle. ...,- G1 the cell grows/duplication of organelles\...
1,\n\nQuestion: Define the term transpiration an...,Define the term transpiration and explain the ...,- (transpiration is) loss of water vapour from...
2,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...
3,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...
4,\n\nQuestion: Explain the use of karyotyping i...,Explain the use of karyotyping in human geneti...,- Definition and construction of karyotypes:\n...
...,...,...,...
239,\n\nQuestion: All parts of the body change the...,All parts of the body change the composition o...,- a. higher nitrogen/urea as blood enters neph...
240,Question: Outline the process of spermatogenes...,Outline the process of spermatogenesis in huma...,- production of sperm/spermatozoa in the teste...
241,\n\nQuestion: Explain the structure and functi...,Explain the structure and function of the plac...,- embryonic/disc shaped structure that nourish...
242,\n\nQuestion: Nitrogen is part of many importa...,Nitrogen is part of many important substances ...,- DNA is transcribed AND mRNA is translated\n-...


In [ ]:
# It captures numbers that are followed by either 'mark' or 'marks' within brackets
pattern = r'\[\s*(\d+)\s*mark'

# Extract the number and assign it to a new column 'Number of Marks'
df['Number of Marks'] = df['Question'].str.extract(pattern)

In [ ]:
df['Number of Marks'] = df['Number of Marks'].astype('Int64')

In [ ]:
df

,p2 question and answers,Question,Markscheme,Number of Marks
0,Question: Outline the various stages of the ce...,Outline the various stages of the cell cycle. ...,- G1 the cell grows/duplication of organelles\...,4
1,\n\nQuestion: Define the term transpiration an...,Define the term transpiration and explain the ...,- (transpiration is) loss of water vapour from...,9
2,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,5
3,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...,5
4,\n\nQuestion: Explain the use of karyotyping i...,Explain the use of karyotyping in human geneti...,- Definition and construction of karyotypes:\n...,8
...,...,...,...,...
239,\n\nQuestion: All parts of the body change the...,All parts of the body change the composition o...,- a. higher nitrogen/urea as blood enters neph...,7
240,Question: Outline the process of spermatogenes...,Outline the process of spermatogenesis in huma...,- production of sperm/spermatozoa in the teste...,5
241,\n\nQuestion: Explain the structure and functi...,Explain the structure and function of the plac...,- embryonic/disc shaped structure that nourish...,9
242,\n\nQuestion: Nitrogen is part of many importa...,Nitrogen is part of many important substances ...,- DNA is transcribed AND mRNA is translated\n-...,4


In [ ]:
unique_value_counts = df['Number of Marks'].value_counts(dropna=True)

print(unique_value_counts)

Number of Marks
8    65
4    63
6    39
5    35
7    10
9     8
Name: count, dtype: Int64


In [ ]:
# Number of samples you want from each category
n_samples = 9

# Function to sample or duplicate entries if there are not enough in the category
def sample_or_duplicate(group):
    if len(group) < n_samples:
        return group.sample(n_samples, replace=True)  # Allow duplicate if not enough entries
    return group.sample(n_samples)

# Sample 9 questions from each unique value of 'Number of Marks'
sampled_df = df.groupby('Number of Marks').apply(sample_or_duplicate).reset_index(drop=True)

display(sampled_df)


,p2 question and answers,Question,Markscheme,Number of Marks
0,Question: Draw a labelled diagram of Escherich...,Draw a labelled diagram of Escherichia coli as...,- Award [1] for each structure clearly drawn a...,4
1,"Question: Outline pollination, fertilization a...","Outline pollination, fertilization and seed di...",- pollination is the transfer of pollen to the...,4
2,Question: Draw a labelled diagram of a mature ...,Draw a labelled diagram of a mature sperm cell...,- Award [1] for each of the following clearly ...,4
3,Question: Draw a labelled diagram of the struc...,Draw a labelled diagram of the structure of a ...,- Award [1] for each of the following clearly ...,4
4,\n\nQuestion: Nitrogen is part of many importa...,Nitrogen is part of many important substances ...,- DNA is transcribed AND mRNA is translated\n-...,4
5,Question: Describe the relationship between ge...,"Describe the relationship between genes, polyp...",- gene is a sequence of DNA bases\n- DNA/gene ...,4
6,\n\nQuestion: Outline the hormonal control of ...,Outline the hormonal control of the process of...,- level of progesterone decreases (drastically...,4
7,Question: Draw a labelled diagram of a eukaryo...,Draw a labelled diagram of a eukaryotic plant ...,- Cell wall shown with two continuous lines to...,4
8,Question: Describe the process of blood clotti...,Describe the process of blood clotting. [4 marks],- clotting factor released by platelets/damage...,4
9,\n\nQuestion: Outline two factors that affect ...,Outline two factors that affect the rate of ph...,- (increase in) light (intensity) increases ra...,5


In [ ]:

# Filter to get only the rows where 'Number of Marks' is 5
df_five_marks = df[df['Number of Marks'] == 5].reset_index(drop=True)

# This new DataFrame df_five_marks contains only the questions with 5 marks
display(df_five_marks)

,p2 question and answers,Question,Markscheme,Number of Marks
0,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,5
1,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...,5
2,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the structure ...,- phospholipid bilayer – with head and tails\n...,5
3,\n\nQuestion: Outline two factors that affect ...,Outline two factors that affect the rate of ph...,- (increase in) light (intensity) increases ra...,5
4,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the structure ...,- Award [1] for each structure clearly drawn a...,5
5,\n\nQuestion: Describe the process of endocyto...,Describe the process of endocytosis. [5 marks],- plasma membrane encloses/engulfs solid parti...,5
6,"Question: Outline, with an example, the proces...","Outline, with an example, the process of exocy...",- vesicles carry material to plasma membrane\n...,5
7,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the ultrastruc...,- Award [1] for each structure clearly drawn a...,5
8,Question: Describe how plants carry out gas ex...,Describe how plants carry out gas exchange in ...,- gases/O2 and CO2 enter/exit the leaf through...,5
9,\n\nQuestion: Outline the causes and consequen...,Outline the causes and consequences of the enh...,- burning of (fossil) fuels/coal/oil/gas relea...,5


In [ ]:
df_five_marks['Question'][2]

'Draw a labelled diagram to show the structure of the plasma membrane. [5 marks]'

In [ ]:
from openai import OpenAI
import os
client = OpenAI(api_key=api_key)

In [ ]:
def rephrase_into_student_question(question,model_):
    example_question = "Outline the effect of temperature and substrate concentration on the activity of enzymes. [5 marks]"
    example_answer="What is the effect of temperature and substrate concentration on the activity of enzymes?"

    # Define the new task for the assistant
    prompt = (
        f"Example Question:{example_question}"
        f"Example Answer:{example_answer}"
        "Rephrase the question below in a similar way."
        f"Question: {question}"
        "Answer:"
    )

    # Update system message to match the new task
    system_message = "You are an assistant that rephrases questions into questions that could come from a student. Ensure that the questions don't have to do with drawing anything."

    # Use OpenAI API to generate the response
    response = client.chat.completions.create(
        model=model_,  # Adjust the model as per availability or requirements
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,  # Adjust temperature as needed to balance creativity and accuracy
        max_tokens=100,  # Adjust max tokens if more are needed for longer answers
        top_p=1
    )

    complete_response = response.choices[0].message.content.strip()
    # Return the generated response
    return complete_response.strip()


In [ ]:
rephrase_into_student_question(df_five_marks['Question'][2],'gpt-4')

'Can you explain the structure of the plasma membrane?'

In [ ]:
from tqdm import tqdm
df_five_marks['Qualitative benchmark question']=[rephrase_into_student_question(question,'gpt-4') for question in tqdm(df_five_marks['Question'])]

100%|██████████| 35/35 [01:06<00:00,  1.89s/it]


In [ ]:
df_five_marks

,p2 question and answers,Question,Markscheme,Number of Marks,Qualitative benchmark question
0,Question: Describe the characteristics of stem...,Describe the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,5,What are the characteristics of stem cells tha...
1,\n\nQuestion: Outline a technique of gene tran...,Outline a technique of gene transfer resulting...,- gene transfer takes a gene from one species/...,5,Can you explain a technique of gene transfer t...
2,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the structure ...,- phospholipid bilayer – with head and tails\n...,5,Can you explain the structure of the plasma me...
3,\n\nQuestion: Outline two factors that affect ...,Outline two factors that affect the rate of ph...,- (increase in) light (intensity) increases ra...,5,What are two factors that influence the rate o...
4,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the structure ...,- Award [1] for each structure clearly drawn a...,5,Can you explain the structure of a sarcomere?
5,\n\nQuestion: Describe the process of endocyto...,Describe the process of endocytosis. [5 marks],- plasma membrane encloses/engulfs solid parti...,5,Can you explain the process of endocytosis?
6,"Question: Outline, with an example, the proces...","Outline, with an example, the process of exocy...",- vesicles carry material to plasma membrane\n...,5,Can you explain the process of exocytosis with...
7,Question: Draw a labelled diagram to show the ...,Draw a labelled diagram to show the ultrastruc...,- Award [1] for each structure clearly drawn a...,5,Can you describe the ultrastructure of Escheri...
8,Question: Describe how plants carry out gas ex...,Describe how plants carry out gas exchange in ...,- gases/O2 and CO2 enter/exit the leaf through...,5,How do plants perform gas exchange in the leaves?
9,\n\nQuestion: Outline the causes and consequen...,Outline the causes and consequences of the enh...,- burning of (fossil) fuels/coal/oil/gas relea...,5,Can you explain the causes and consequences of...


In [ ]:
df_qual_quest=df_five_marks[['Qualitative benchmark question','Markscheme']]
df_qual_quest

,Qualitative benchmark question,Markscheme
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...
1,Can you explain a technique of gene transfer t...,- gene transfer takes a gene from one species/...
2,Can you explain the structure of the plasma me...,- phospholipid bilayer – with head and tails\n...
3,What are two factors that influence the rate o...,- (increase in) light (intensity) increases ra...
4,Can you explain the structure of a sarcomere?,- Award [1] for each structure clearly drawn a...
5,Can you explain the process of endocytosis?,- plasma membrane encloses/engulfs solid parti...
6,Can you explain the process of exocytosis with...,- vesicles carry material to plasma membrane\n...
7,Can you describe the ultrastructure of Escheri...,- Award [1] for each structure clearly drawn a...
8,How do plants perform gas exchange in the leaves?,- gases/O2 and CO2 enter/exit the leaf through...
9,Can you explain the causes and consequences of...,- burning of (fossil) fuels/coal/oil/gas relea...


In [ ]:
df_qual_quest.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/qualitative_data_questions/qualitative_questions.csv')